In [ ]:
pip install -U ckiptagger[tfgpu,gdown]

In [ ]:
pip install opencc-python-reimplemented

In [ ]:
#mount gdrive
from google.colab import drive
import os
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#solving the environment with ckiptagger
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import pandas as pd
import numpy as np
import re
from opencc import OpenCC
ws=WS("/content/gdrive/My Drive/data", disable_cuda=False)

In [ ]:
review=pd.read_json('/content/gdrive/My Drive/Colab Notebooks/Dataset/Original_dataset/reviews.json')

pos_table = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/Data_preprocess/External_resource/pos_table.xlsx',sheet_name='Sheet1')
neg_table = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/Data_preprocess/External_resource/neg_table.xlsx',sheet_name='Sheet1')

In [ ]:
for i in range(len(review)):
  tmp=review['content'][i].replace('\n', '')
  cop=re.compile("[^\u4e00-\u9fa5]")
  cc = OpenCC('s2t')
  review.loc[review.content==review['content'][i], 'content'] =cc.convert(cop.sub('', tmp))
  if review['content'][i].strip()=='':
    review=review.drop(index=i)

review=review.reset_index(drop=True)

Here I merge two dict. in one, and for the neg_dict, i time it with -1 to make it <0 that denotes the negative sentiment score

In [ ]:
pos_dict = dict(zip(list(pos_table.posword),list(pos_table.score)))
neg_dict = dict(zip(list(neg_table.negword),map(lambda a:a*(0-1),list(neg_table.score)) ))
sentiment_dict={**pos_dict,**neg_dict}

Here i also store the original content and segment content for the use of calculating sentiment score later.

In [ ]:
review['original_content']=''
review['segment_content']=''

tmp_corpus=[]
for i in range(len(review)):
  tmp_corpus.append(review['content'][i])
tmp_corpus=ws(tmp_corpus)

cut_corpus=[]
for j in tmp_corpus:
  cut_corpus.append(' '.join(j))

for k in range(len(review)):
  review.at[k,'original_content']=review['content'][k]
  review.at[k,'segment_content']=ws([review['content'][k]])
  review.at[k,'content']=cut_corpus[k]

The sentiment score is calculated as the following step


```
# first cut the sentence into segments
# second, if the token exists in the sentiment dict. , add the score with its corresponding score, and record the count of word exists simultaneously.
# finally, return (score/countword)/9 as the review's sentiment score.
# (9 is the highest score above all, and i devide it to regularize the score between the bound [1,-1])
```



In [ ]:
def get_sentiment(sent):
  tokens = ws([sent])
  score=0
  countword=0

  for w in range(len(tokens[0])):
    if tokens[0][w] in sentiment_dict:
      score+=sentiment_dict[tokens[0][w]]
      countword+=1
  if countword!=0:
    return (score/countword)/9
  else:
    return 0
  


In [ ]:
review['sentiment']=0
for i in range(len(review)):
  review.at[i,'sentiment']=get_sentiment(review['original_content'][i])

In [ ]:
review.to_csv('/content/gdrive/My Drive/Colab Notebooks/Dataset/Output_dataset/review_w_sentiment.csv',encoding='utf-8',index=False)